<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Treelite" data-toc-modified-id="Treelite-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Treelite</a></span></li></ul></div>

# Treelite

https://github.com/dmlc/treelite

```bash
git clone --recursive https://github.com/dmlc/treelite.git
cd treelite

mkdir build
cd build

cmake .. -DCMAKE_CXX_COMPILER=g++-7 -DCMAKE_C_COMPILER=gcc-7

make

cd ../python
python setup.py install
```

In [1]:
from sklearn.datasets import load_boston

X, y = load_boston(return_X_y=True)
print('dimensions of X = {}'.format(X.shape))
print('dimensions of y = {}'.format(y.shape))

dimensions of X = (506, 13)
dimensions of y = (506,)


In [2]:
import xgboost

params = {
    'max_depth':3, 'eta':1, 'silent':1,
    'objective':'reg:linear',
    'eval_metric':'rmse'
}

dtrain = xgboost.DMatrix(X, label=y)
bst = xgboost.train(params, dtrain, 20, [(dtrain, 'train')])
bst

[0]	train-rmse:3.89049
[1]	train-rmse:3.38204
[2]	train-rmse:3.10513
[3]	train-rmse:2.84322
[4]	train-rmse:2.6058
[5]	train-rmse:2.45425
[6]	train-rmse:2.29526
[7]	train-rmse:2.1792
[8]	train-rmse:2.09359
[9]	train-rmse:1.96872
[10]	train-rmse:1.93416
[11]	train-rmse:1.83528
[12]	train-rmse:1.7875
[13]	train-rmse:1.71018
[14]	train-rmse:1.64747
[15]	train-rmse:1.57359
[16]	train-rmse:1.49626
[17]	train-rmse:1.43896
[18]	train-rmse:1.37123
[19]	train-rmse:1.30187


In [3]:
import treelite

model = treelite.Model.from_xgboost(bst)
model

In [4]:
toolchain = 'gcc'

model.export_lib(toolchain=toolchain, libpath='./mymodel.dylib', verbose=True)

[18:23:30] /Users/mingyuliu/treelite/src/compiler/ast_native.cc:43: Using ASTNativeCompiler
[18:23:30] /Users/mingyuliu/treelite/src/compiler/ast/split.cc:23: Parallel compilation disabled; all member trees will be dumped to a single source file. This may increase compilation time and memory usage.
[18:23:30] /Users/mingyuliu/treelite/src/c_api/c_api.cc:286: Code generation finished. Writing code to files...
[18:23:30] /Users/mingyuliu/treelite/src/c_api/c_api.cc:291: Writing file recipe.json...
[18:23:30] /Users/mingyuliu/treelite/src/c_api/c_api.cc:291: Writing file main.c...
[18:23:30] /Users/mingyuliu/treelite/src/c_api/c_api.cc:291: Writing file header.h...
[18:23:30] /Users/mingyuliu/anaconda3/lib/python3.6/site-packages/treelite-0.9-py3.6.egg/treelite/contrib/util.py:97: Compiling sources files in directory /var/folders/b6/fy5cl70s7nq6g7275rkdpd6m3967fj/T/tmpr3a200gz into object files (*.o)...
[18:23:30] /Users/mingyuliu/anaconda3/lib/python3.6/site-packages/treelite-0.9-py3.6.e

In [5]:
import treelite.runtime

predictor = treelite.runtime.Predictor('./mymodel.dylib', verbose=True)
predictor

[18:23:30] /Users/mingyuliu/anaconda3/lib/python3.6/site-packages/treelite-0.9-py3.6.egg/treelite/runtime/treelite_runtime/predictor.py:283: Dynamic shared library /Users/mingyuliu/Desktop/treelite/mymodel.dylib has been successfully loaded into memory


In [8]:
batch = treelite.runtime.Batch.from_npy2d(X)

In [12]:
%%timeit
bst.predict(dtrain)

69.8 µs ± 1.65 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [13]:
%%timeit
predictor.predict(batch)

51.7 µs ± 5.06 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
